# OCC daily EDA

EDA de dataset por día

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession \
    .builder \
    .appName("Learning Spark") \
    .getOrCreate()

date_str = "20231002"

base_dir = f"harvester/occ/{date_str}/"
df = spark.read.json(f"{base_dir}/occ-{date_str}.jsonl.gz")
categories = spark.read.json(f"{base_dir}/occ-{date_str}-categories.json")
subcategories = spark.read.json(f"{base_dir}/occ-{date_str}-subcategories.json")

In [3]:
df.printSchema()

root
 |-- Job:value: string (nullable = true)
 |-- __typename: string (nullable = true)
 |-- applied: boolean (nullable = true)
 |-- autoinclusion: struct (nullable = true)
 |    |-- __typename: string (nullable = true)
 |    |-- autoinclusion: boolean (nullable = true)
 |    |-- source: string (nullable = true)
 |-- bullets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- __ref: string (nullable = true)
 |-- category: struct (nullable = true)
 |    |-- __ref: string (nullable = true)
 |-- company: struct (nullable = true)
 |    |-- __typename: string (nullable = true)
 |    |-- confidential: boolean (nullable = true)
 |    |-- logoUrl: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- namePretty: string (nullable = true)
 |    |-- profile: struct (nullable = true)
 |    |    |-- __ref: string (nullable = true)
 |    |-- rel: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- contact: struct (nullable =

In [4]:
df.show(1, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Número de Registros

In [5]:
#dfu = df.dropDuplicates(["id"])
dfu = df.dropDuplicates(["id"]).filter(df.redirect.type < 2)
record_count = dfu.count()
print(f"Registros totales: {df.count()}, id unicos: {record_count}")

Registros totales: 154198, id unicos: 76569


### Por tipo de redireccionamiento

- 0: No redireccionadas
- 1: ? (¿redireccionamiento pagado?)
- 2: Redir fuera del sitio (¿son agregadas?)

In [6]:
dfu.groupby("redirect.type").count().show()

+----+-----+
|type|count|
+----+-----+
|   0|73341|
|   1| 3228|
+----+-----+



### Por tipo de anuncio
Tablas para todas las vacantes y paras las únicas. 

Notar que las Premium parecen tener mas duplicadas, puede ser por el proceso de scraping cuando aparecen en varios SERP

In [7]:
df.groupby("jobType").count().show()

+--------+------+
| jobType| count|
+--------+------+
| PREMIUM|  3499|
|STANDOUT| 27733|
| CLASSIC|122966|
+--------+------+



In [8]:
dfu.groupby("jobType").count().show()

+--------+-----+
| jobType|count|
+--------+-----+
| PREMIUM| 2603|
|STANDOUT|27394|
| CLASSIC|46572|
+--------+-----+



### Tipo de Anuncio / redireccionamiento

CLASSIC / Redir=2 => Suponemos que son las agregadas

In [9]:
dfu.groupby("jobType").pivot("redirect.type").count().show()

+--------+-----+----+
| jobType|    0|   1|
+--------+-----+----+
| PREMIUM| 2561|  42|
|STANDOUT|27055| 339|
| CLASSIC|43725|2847|
+--------+-----+----+



#### Proporciones por JobType

In [11]:
pdf = dfu.groupby("jobType").pivot("redirect.type").count().toPandas().fillna(0).set_index('jobType')
#pdf.columns = ['NoRedir', 'Redir1', 'Redir2']
pdf.div(pdf.sum(axis=1), axis=0)

,0,1
jobType,,
PREMIUM,0.983865,0.016135
STANDOUT,0.987625,0.012375
CLASSIC,0.938869,0.061131


#### Proporciones por tipo de redir

Esto demostraría que las Redir2 son agregadas no pagadas ya que practicamente todas son clásicas

In [12]:
pdf.div(pdf.sum(axis=0), axis=1)

,0,1
jobType,,
PREMIUM,0.034919,0.013011
STANDOUT,0.368893,0.105019
CLASSIC,0.596188,0.881970


#### Proporciones del total

In [13]:
pdf.div(pdf.sum().sum())

,0,1
jobType,,
PREMIUM,0.033447,0.000549
STANDOUT,0.353341,0.004427
CLASSIC,0.571054,0.037182


## Categorias

### Totales

In [14]:
cat_count = dfu.groupby("category").count().orderBy(F.col("count").desc())
split_col = F.split(cat_count["category.__ref"].cast("String"), ":")
cat_count = cat_count.withColumn("category_id", split_col.getItem(1).cast("INT"))
cat_count = cat_count.join(categories.select("id", "description"), cat_count.category_id == categories.id, how="inner")
cat_count = cat_count.select(["category_id", "description", "count"]).orderBy(F.col("count").desc())

In [15]:
cat_count = cat_count.withColumn("pct", F.col("count") / record_count)
cat_count.show(cat_count.count(), truncate=False)

+-----------+------------------------------------------------+-----+---------------------+
|category_id|description                                     |count|pct                  |
+-----------+------------------------------------------------+-----+---------------------+
|19         |Ventas                                          |16239|0.21208321905732086  |
|17         |Tecnologías de la Información - Sistemas        |8237 |0.10757617312489388  |
|4          |Contabilidad - Finanzas                         |8190 |0.1069623476864005   |
|10         |Logística - Transporte - Distribución - Almacén |6537 |0.08537397641343103  |
|1          |Administrativo                                  |6508 |0.0849952330577649   |
|11         |Manufactura - Producción - Operación            |5429 |0.07090336820384228  |
|9          |Ingeniería                                      |4781 |0.06244041322206115  |
|13         |Recursos humanos                                |3773 |0.04927581658373493  |

### Proporciones por tipo de Redir

In [17]:
split_col = F.split(dfu["category.__ref"].cast("String"), ":")
cat_count_redir = dfu.withColumn("category_id", split_col.getItem(1).cast("INT"))
cat_count_redir = cat_count_redir.groupby("category_id").pivot("redirect.type").count()
cat_count_redir = cat_count_redir.join(categories.select("id", "description"), cat_count_redir.category_id == categories.id, how="inner")
cat_count_redir = cat_count_redir.select(["description", "0", "1", "2"]).sort(F.col("0").desc()).toPandas().fillna(0)
cat_count_redir.columns = ["Category", "NoRedir", "Redir1"]#, "Redir2"]
cat_count_redir = cat_count_redir.set_index("Category")
cat_count_redir = cat_count_redir.div(cat_count_redir.sum(axis=1), axis=0)
cat_count_redir

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `2` cannot be resolved. Did you mean one of the following? [`0`, `1`, `id`, `category_id`, `description`].;
'Project [description#93, 0#2538L, 1#2539L, '2]
+- Join Inner, (category_id#2337 = cast(id#94 as int))
   :- Project [category_id#2337, __pivot_count(1) AS count AS `count(1) AS count`#2537[0] AS 0#2538L, __pivot_count(1) AS count AS `count(1) AS count`#2537[1] AS 1#2539L]
   :  +- Aggregate [category_id#2337], [category_id#2337, pivotfirst(type#2530L, count(1) AS count#2531L, 0, 1, 0, 0) AS __pivot_count(1) AS count AS `count(1) AS count`#2537]
   :     +- Aggregate [category_id#2337, redirect#28.type], [category_id#2337, redirect#28.type AS type#2530L, count(1) AS count(1) AS count#2531L]
   :        +- Project [Job:value#8, __typename#9, applied#10, autoinclusion#11, bullets#12, category#13, company#14, contact#15, dates#16, description#17, education#18, friendlyId#19, googleForJobs

#### Categorías en las que OCC es débil

Mayor proporción de agregadas

In [ ]:
cat_count_redir.sort_values("Redir2", ascending=False)

## Subcategorías

In [ ]:
subcat_count = dfu.groupby("subcategory").count().orderBy(F.col("count").desc())
split_col = F.split(subcat_count["subcategory.__ref"].cast("String"), ":")
subcat_count = subcat_count.withColumn("subcategory_id", split_col.getItem(1).cast("INT"))
subcat_count = subcat_count.join(subcategories.select("id", "description"), subcat_count.subcategory_id == subcategories.id, how="inner")
subcat_count.orderBy(F.col("count").desc()).show(25, truncate=False)

In [ ]:
split_col = F.split(dfu["subcategory.__ref"].cast("String"), ":")
subcat_count_redir = dfu.withColumn("subcategory_id", split_col.getItem(1).cast("INT"))
subcat_count_redir = subcat_count_redir.groupby("subcategory_id").pivot("redirect.type").count()
subcat_count_redir = subcat_count_redir.join(subcategories.select("id", "description"), subcat_count_redir.subcategory_id == subcategories.id, how="inner")
subcat_count_redir = subcat_count_redir.select(["description", "0", "1", "2"]).sort(F.col("0").desc()).toPandas().fillna(0)
subcat_count_redir.columns = ["subcategory", "NoRedir", "Redir1", "Redir2"]
subcat_count_redir = subcat_count_redir.set_index("subcategory")
subcat_count_redir = subcat_count_redir.div(subcat_count_redir.sum(axis=1), axis=0)
pd.set_option('display.max_rows', None)
subcat_count_redir

In [ ]:
subcat_count_redir.sort_values("Redir2", ascending=False)

## Google For Jobs

- Primer campo: si la oferta está disponible para G4J
- Segundo campo: (¿?) si la oferta ya está indexada por G4J

In [ ]:
dfu.groupby("googleForJobs").count().sort(F.col("count").desc()).show(truncate=False)

## Ubicación

### Estados

#### Distribución de vacantes por estado con suma acumulada

In [ ]:
dfu_loc = dfu.select("id", F.explode("location.locations").alias("loc_data"))
state_count = dfu_loc.groupby("loc_data.state.description").count().sort(F.col("count").desc())
state_count_p = state_count.withColumn("perc", F.col("count") / record_count).orderBy(F.col("perc").desc())
window = Window.orderBy(F.col("perc").desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
state_count_p = state_count_p.withColumn("cumsum", F.sum(F.col("perc")).over(window))
state_count_p.show(33)

#### Vacantes por estado divididas por tipo Redir, con histograma para Redir2

In [ ]:
state_count = dfu.withColumn("loc_data", F.explode("location.locations"))\
                .groupby("loc_data.state.description")\
                .pivot("redirect.type").count()\
                .withColumn("total", F.col("0") + F.col("1") + F.col("2"))\
                .withColumn("NoRedirPct", F.col("0") / F.col("total"))\
                .withColumn("Redir1Pct", F.col("1") / F.col("total"))\
                .withColumn("Redir2Pct", F.col("2") / F.col("total"))\
                .sort(F.col("total").desc())

In [ ]:
state_count.show()

In [ ]:
state_count_pd = state_count.toPandas()
print(state_count_pd["Redir2Pct"].describe())
state_count_pd["Redir2Pct"].hist(bins=15)

#### Descripción (granular)

In [ ]:
loc_count = dfu.groupby("location.description").count().sort(F.col("count").desc())
loc_count_p = loc_count.withColumn("perc", F.col("count") / record_count).orderBy(F.col("perc").desc())
window = Window.orderBy(F.col("perc").desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
loc_count_p = loc_count_p.withColumn("cumsum", F.sum(F.col("perc")).over(window))
print(f"Número de location.description: {loc_count_p.count()}")
loc_count_p.show(30, truncate=False)

In [ ]:
loc_count_p.toPandas()["cumsum"].plot()

## Compañias

Basadas en la url. 

NULL ==> Confidenciales

In [ ]:
company_count = dfu.groupby("company.url").count().sort(F.col("count").desc())
print(f"Número de URLs de compañias: {company_count.count()}")
company_count.show(25, truncate=False)

#### Proporción de vacantes por cia y suma acumulada

In [ ]:
ccp = company_count.withColumn("perc", F.col("count") / record_count).orderBy(F.col("perc").desc())
window = Window.orderBy(F.col("perc").desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
ccp.withColumn("cumsum", F.sum(F.col("perc")).over(window)).show(30, truncate=False)

### Excluyendo confidenciales

In [ ]:
ccp_noconf = company_count.where("url is not null")
nconf_records = ccp_noconf.agg(F.sum("count")).collect()[0][0]
ccp_noconf = ccp_noconf.withColumn("perc", F.col("count") / nconf_records).orderBy(F.col("perc").desc())
window = Window.orderBy(F.col("perc").desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
ccp_noconf.withColumn("cumsum", F.sum(F.col("perc")).over(window)).show(30, truncate=False)

### Confidenciales

La URL null son confidenciales

In [ ]:
dfu.select(["company.url", "company.name"]).where("company.confidential=TRUE").count()

In [ ]:
dfu.select("company.url").where("company.confidential=TRUE").distinct().count()

### URLs con minusculas -> agregadas

Al parecer las vacantes que tienen company.url en minusculas pero igual a otra URL son de vacantes agregadas

In [ ]:
dfu.where("company.url='empleos/bolsa-de-trabajo-Manpower--S-A--de-C-V-/'").count()

In [ ]:
company_count = dfu.where("redirect.type != 2").groupby("company.url").count().sort(F.col("count").desc())
company_count.show(truncate=False)

In [ ]:
company_count = dfu.where("redirect.type == 2").groupby("company.url").count().sort(F.col("count").desc())
company_count.show(truncate=False)

Hay que revisar qué nos dicen estos datos para inferir mejor 

In [ ]:
dfu.where("redirect.isRedirected == TRUE").where("autoinclusion.autoinclusion == TRUE").count()

In [ ]:
dfu.where("redirect.isRedirected == TRUE").where("autoinclusion.autoinclusion == FALSE").count()

In [ ]:
dfu.where("redirect.isRedirected == FALSE").where("autoinclusion.autoinclusion == TRUE").count()

In [ ]:
dfu.where("redirect.isRedirected == FALSE").where("autoinclusion.autoinclusion == FALSE").count()

In [ ]:
dfu.where("redirect.isRedirected == TRUE")\
    .where("autoinclusion.autoinclusion == TRUE")\
    .select(["scraped_url", "redirect.externalUrl"])\
    .show(truncate=False)

## Salarios

In [ ]:
dfu.select(["salary.from", "salary.to"]).summary().show()

Casi todas las Redir2 (agregadas) no tienen salario, vs 1/3 de las pagadas:

In [ ]:
dfu.withColumn("has_salary", F.col("salary.from") != 0).groupby("redirect.type").pivot("has_salary").count().show()

Al parecer si alguno de los dos campos de salario es mayor a 0, el otro también lo es

In [ ]:
salary = dfu.select(["salary.from", "salary.to"]).where("salary.from > 0 or salary.to > 0")
salary = salary.withColumn("avg", (F.col("from") + F.col("to")) / 2)
salary.summary().show()

In [ ]:
#seleccionar rangos para eliminar outliers - los percentiles son ajustados a algo que permita comparar con CT
outlier_limits = salary.select(F.percentile_approx("avg", [0.009, 0.9995]))
outlier_min, outlier_max = outlier_limits.collect()[0][0]
outlier_limits.show()

In [ ]:
salary = salary.where(f"avg >= {outlier_min} and avg <= {outlier_max}")
salary.summary().show()

Esto es solo para extraer jobids que va a ser usados despues

In [ ]:
with open("occ_jobids_1.txt", "w") as fo:
    [fo.write(f"{y}\n") for y in dfu.select("id").sort(F.col("id").cast("int").desc()).limit(10000).rdd.flatMap(lambda x: x).collect()]